# Нейронные сети
__Суммарное количество баллов: 10__

В этом домашнем задании можно использовать готовые классы библиотек PyTorch, Keras, TensorFlow.

### Задание 1 (4 балла)

Создайте сеть из трех сверточных слоев по 8 сверток 3х3 и двух полносвязных слоев по 64 нейрона. 

Обучите сеть на датасете mnist с тремя разными функциями активации в слоях (sigmoid, tanh, ReLU).

In [2]:
import torch
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
import torchvision.datasets

In [3]:
df = pd.read_csv('mnist.csv')
y = df['label']
X = df.drop('label', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = torch.Tensor(X_train.values).reshape((X_train.shape[0], 28, 28))
X_test = torch.Tensor(X_test.values).reshape((X_test.shape[0], 28, 28))
y_train = torch.LongTensor(y_train.values)
y_test = torch.LongTensor(y_test.values)

In [4]:
X_train = X_train.unsqueeze(1).float()
X_test = X_test.unsqueeze(1).float()

## Sigmoid

In [116]:
class nnet_sigmoid(torch.nn.Module):
    def __init__(self):
        super(nnet_sigmoid, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)
        self.act1 = torch.nn.Sigmoid()
        self.conv2 = torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1)
        self.act2 = torch.nn.Sigmoid()
        self.conv3 = torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1)
        self.act3 = torch.nn.Sigmoid()
        self.fc1 = torch.nn.Linear(28*28*8, 64)
        self.act4 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(64, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.conv3(x)
        x = self.act3(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.act4(x)
        x = self.fc2(x)
        return x

In [118]:
nn = nnet_sigmoid()
loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(nn.parameters(), lr=1.0e-3)
for epoch in range(10):
    indxies = np.random.permutation(7500)
    batch_size = 50
    n_bathes = 7500//batch_size
    for i in range(n_bathes):
        interval = indxies[batch_size * i : batch_size * (i + 1)]
        b = X_train[interval]
        opt.zero_grad()
        preds = nn.forward(b)
        loss_value = loss(preds, y_train[interval])
        loss_value.backward()
        opt.step()
    test_preds = nn.forward(X_test)
    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data.cpu()
    print(epoch, ':', accuracy)

0 : tensor(0.1028)
1 : tensor(0.1060)
2 : tensor(0.6772)
3 : tensor(0.8252)
4 : tensor(0.8632)
5 : tensor(0.8852)
6 : tensor(0.9012)
7 : tensor(0.9044)
8 : tensor(0.9164)
9 : tensor(0.9180)


## Tanh

In [58]:
class nnet_tanh(torch.nn.Module):
    def __init__(self):
        super(nnet_tanh, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)
        self.act1 = torch.nn.Tanh()
        self.conv2 = torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1)
        self.act2 = torch.nn.Tanh()
        self.conv3 = torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1)
        self.act3 = torch.nn.Tanh()
        self.fc1 = torch.nn.Linear(28*28*8, 64)
        self.act4 = torch.nn.Tanh()
        self.fc2 = torch.nn.Linear(64, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.conv3(x)
        x = self.act3(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.act4(x)
        x = self.fc2(x)
        return x

In [73]:
nn = nnet_tanh()
loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(nn.parameters(), lr=1.0e-3)
for epoch in range(15):
    indxies = np.random.permutation(7500)
    batch_size = 50
    n_bathes = 7500//batch_size
    for i in range(n_bathes):
        interval = indxies[batch_size * i : batch_size * (i + 1)]
        b = X_train[interval]
        opt.zero_grad()
        preds = nn.forward(b)
        loss_value = loss(preds, y_train[interval])
        loss_value.backward()
        opt.step()
    test_preds = nn.forward(X_test)
    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data.cpu()
    print(epoch, ':', accuracy)

0 : tensor(0.9228)
1 : tensor(0.9500)
2 : tensor(0.9532)
3 : tensor(0.9680)
4 : tensor(0.9700)
5 : tensor(0.9740)
6 : tensor(0.9744)
7 : tensor(0.9744)
8 : tensor(0.9748)
9 : tensor(0.9752)
10 : tensor(0.9756)
11 : tensor(0.9776)
12 : tensor(0.9764)
13 : tensor(0.9780)
14 : tensor(0.9772)


## ReLU

In [50]:
class nnet_relu(torch.nn.Module):
    def __init__(self):
        super(nnet_relu, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)
        self.act1 = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1)
        self.act2 = torch.nn.ReLU()
        self.conv3 = torch.nn.Conv2d(in_channels=8, out_channels=1, kernel_size=3, padding=1)
        self.act3 = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(28*28*1, 64)
        self.act4 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(64, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.conv3(x)
        x = self.act3(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.act4(x)
        x = self.fc2(x)
        return x

In [74]:
nn = nnet_relu()
loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(nn.parameters(), lr=1.0e-3)
for epoch in range(10):
    indxies = np.random.permutation(7500)
    batch_size = 50
    n_bathes = 7500//batch_size
    for i in range(n_bathes):
        interval = indxies[batch_size * i : batch_size * (i + 1)]
        b = X_train[interval]
        opt.zero_grad()
        preds = nn.forward(b)
        loss_value = loss(preds, y_train[interval])
        loss_value.backward()
        opt.step()
    test_preds = nn.forward(X_test)
    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data.cpu()
    print(epoch, ':', accuracy)

0 : tensor(0.9108)
1 : tensor(0.9272)
2 : tensor(0.9284)
3 : tensor(0.9444)
4 : tensor(0.9560)
5 : tensor(0.9396)
6 : tensor(0.9512)
7 : tensor(0.9560)
8 : tensor(0.9600)
9 : tensor(0.9512)


### Задание 2 (6 баллов)

Достигните точности 87% на test датасете notMNIST.

Архитектура сети может быть любая. Можно использовать Data Augmentation.

In [3]:
class nnet(torch.nn.Module):
    def __init__(self):
        super(nnet, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, padding=1)
        self.act1 = torch.nn.Tanh()
        self.conv2 = torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1)
        self.act2 = torch.nn.Tanh()
        self.conv3 = torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1)
        self.act3 = torch.nn.Tanh()
        self.fc1 = torch.nn.Linear(28*28*8, 64)
        self.act4 = torch.nn.Tanh()
        self.fc2 = torch.nn.Linear(64, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.conv3(x)
        x = self.act3(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.act4(x)
        x = self.fc2(x)
        return x

In [4]:
def load_dataset():
    data_path = 'notMNIST_small/'
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=torchvision.transforms.ToTensor()
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=9362,
        num_workers=0,
        shuffle=True
    )
    return train_loader

In [8]:
for n_batch, (feature, label) in enumerate(load_dataset()): 
    if n_batch == 0:
        X_train = feature
        y_train = label
    else:
        X_test = feature
        y_test = label

In [9]:
nn = nnet()
loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(nn.parameters(), lr=1.0e-3)
for epoch in range(20):
    indxies = np.random.permutation(9362)
    batch_size = 40
    n_batches = 9362//batch_size
    for i in range(n_batches):
        interval = indxies[batch_size * i : batch_size * (i + 1)]
        b = X_train[interval]
        opt.zero_grad()
        preds = nn.forward(b)
        loss_value = loss(preds, y_train[interval])
        loss_value.backward()
        opt.step()
    test_preds = nn.forward(X_test)
    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data.cpu()
    print(epoch, ':', accuracy)

0 : tensor(0.8873)
1 : tensor(0.8928)
2 : tensor(0.8986)
3 : tensor(0.9044)
4 : tensor(0.9035)
5 : tensor(0.9038)
6 : tensor(0.9054)
7 : tensor(0.9049)
8 : tensor(0.9075)
9 : tensor(0.9106)
10 : tensor(0.9111)
11 : tensor(0.9105)
12 : tensor(0.9116)
13 : tensor(0.9101)
14 : tensor(0.9093)
15 : tensor(0.9084)
16 : tensor(0.9100)
17 : tensor(0.9076)
18 : tensor(0.9108)
19 : tensor(0.9112)
